In [7]:
import os
 
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)

current directory is : c:\
Directory name is : 


In [10]:
import os
import glob
import pandas

path = 'C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/'
extension = 'csv'
os.chdir(path)
result = [i for i in glob.glob('*.{}'.format(extension))]
print(result)

['F-179.csv', 'F-319.csv', 'F-45.csv', 'G-1220.csv', 'G-1260_Done.csv', 'G-2147.csv', 'G-2866.csv', 'G-3549.csv', 'G-561.csv', 'G-580A.csv', 'G-852_NO_TIDE.csv', 'G-860.csv', 'PB-1680.csv']


# Read file

In [487]:
# Dont do number 4
# Problems with 11, 12
# Next to do: 12

In [488]:
start = '2007-10-01 01:00:00'
end = '2018-06-08 11:00:00'
time_range = pd.date_range(start=start, end=end,freq='H')

In [489]:
len(time_range)

93683

In [681]:
file

'G-860.csv'

In [791]:
file = result[10]
df = pd.read_csv(file)

if 'Code' in df:
    df = df[df.Code == "A"]
file

C:\Users\Christopher\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (3,4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'G-852_NO_TIDE.csv'

In [792]:
df.head()

,Date,Time,Tide_ft,Corrected,Date.1,Time.1,tz_cd,Well_ft,Code,Corrected.1,Date.2,RAIN_FT
0,NaN,NaN,NaN,0.242,7/28/2000,8:24,EDT,2.26,A,0.702,10/10/2005 0:00,0.0
1,NaN,NaN,NaN,0.262,8/11/2000,10:04,EDT,2.22,A,0.662,10/10/2005 0:15,0.0
2,NaN,NaN,NaN,0.282,10/12/2000,17:18,EDT,3.85,A,2.292,10/10/2005 0:30,0.0
3,NaN,NaN,NaN,0.312,1/8/2001,9:47,EST,1.84,A,0.282,10/10/2005 0:45,0.0
4,NaN,NaN,NaN,0.342,4/25/2001,9:32,EDT,1.51,A,-0.048,10/10/2005 1:00,0.0


# Aggregate Tide Water Elevation

In [712]:
df.DateHour.describe()

count             89595
unique            89595
top       2/8/2011 0:00
freq                  1
Name: DateHour, dtype: object

In [747]:
df.head()

,DateHour,TideLevel,WellDateHour,date,time,tz_cd,Well_ft,Code,Corrected,RainDateHour,Date,RAIN_FT,TideDateHour
0,10/1/2007 1:00,-0.098,2007/10/01 01:00,10/1/2007,1:00,EDT,4.43,A,2.872,2007/10/01 01:00,10/1/2007 1:00,0.000000,10/1/2007 1:00
1,10/1/2007 2:00,0.000,2007/10/01 02:00,10/1/2007,2:00,EDT,4.44,A,2.882,2007/10/01 01:00,10/1/2007 1:15,0.000000,10/1/2007 2:00
2,10/1/2007 3:00,-0.273,2007/10/01 03:00,10/1/2007,3:00,EDT,4.44,A,2.882,2007/10/01 01:00,10/1/2007 1:30,0.000000,10/1/2007 3:00
3,10/1/2007 4:00,-0.822,2007/10/01 04:00,10/1/2007,4:00,EDT,4.44,A,2.882,2007/10/01 01:00,10/1/2007 1:45,0.000000,10/1/2007 4:00
4,10/1/2007 5:00,-1.475,2007/10/01 05:00,10/1/2007,5:00,EDT,4.44,A,2.882,2007/10/01 02:00,10/1/2007 2:00,0.005333,10/1/2007 5:00


In [748]:
#%% Create a set format for which to read the string as a datetime variable

df['TideDateHour'] = df.loc[:, 'DateHour']
df['Tide_ft'] = df.loc[:, 'TideLevel']
tide = df.loc[:, ['TideDateHour', 'Tide_ft']]

tide = tide.loc[tide.TideDateHour != '1900/01/00 00:00']

time_format = "%m/%d/%Y %H:%M"

tide['TideDateHour'] = pd.to_datetime(tide.loc[:,'TideDateHour'], format = time_format)

#%% Reorganize the column order
#df = df[['date_hour','date','time','Corrected']]

#%% Setting index to be the date_hour column
tide = tide.set_index('TideDateHour')

#%% Select dates in the range required
data_tide = tide['2007-10-01 01:00:00':'2018-06-08 11:00:00']

#%% Resample data to hourly
data_tide = data_tide.resample('H').mean()

     TideDateHour  Tide_ft
0  10/1/2007 1:00   -0.098
1  10/1/2007 2:00    0.000
2  10/1/2007 3:00   -0.273
3  10/1/2007 4:00   -0.822
4  10/1/2007 5:00   -1.475


In [735]:
data_tide.head()

,Tide_ft
TideDateHour,
2007-10-01 01:00:00,NaN
2007-10-01 02:00:00,NaN
2007-10-01 03:00:00,NaN
2007-10-01 04:00:00,NaN
2007-10-01 05:00:00,NaN


# Aggregate Rain

In [749]:
#%% Create a set format for which to read the string as a datetime variable
rain = df.loc[:, ['RainDateHour', 'RAIN_FT']]


time_format = "%Y/%m/%d %H:%M"

rain['RainDateHour'] = pd.to_datetime(rain['RainDateHour'], format = time_format)

#%% Reorganize the column order
#df = df[['date_hour','date','time','Corrected']]

#%% Setting index to be the date_hour column
rain = rain.set_index('RainDateHour')

print(data_rain.tail())
#%% Select dates in the range required
data_rain = rain['2007-10-01 01:00:00':'2018-06-08 11:00:00']

#%% Resample data to hourly
data_rain = data_rain.resample('H').mean()

data_rain['RAIN_FT'] = data_rain['RAIN_FT'].interpolate()

#%% Write the final clean data to csv
#data_hourly.to_csv('G-1260_Hourly.csv')

                     RAIN_FT
RainDateHour                
2010-06-11 00:00:00      0.0
2010-06-11 01:00:00      0.0
2010-06-11 02:00:00      0.0
2010-06-11 03:00:00      0.0
2010-06-11 04:00:00      0.0


# Aggregate Well

In [750]:
#%% Create a set format for which to read the string as a datetime variable
well = df.loc[:, ['WellDateHour', 'Well_ft']]

well['WellDateHour'] = pd.to_datetime(well['WellDateHour'], format = time_format)

#%% Reorganize the column order
#df = df[['date_hour','date','time','Corrected']]

#%% Setting index to be the date_hour column
well = well.set_index('WellDateHour')

#%% Select dates in the range required
data_narrow = well['2007-10-01 01:00:00':'2018-06-08 11:00:00']

#%% Resample data to hourly
data_well = data_narrow.resample('H').mean()

data_well['Well_ft'] = data_well.Well_ft.interpolate()

#%% Write the final clean data to csv
#data_hourly.to_csv('G-1260_Hourly.csv')

# Merge all of the data into one table and save

In [890]:
file = 'F179_ALL.csv'

In [752]:
len(time_range)

93683

In [891]:
full = pd.DataFrame(index=pd.DatetimeIndex(time_range))

In [892]:
full.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 93683 entries, 2007-10-01 01:00:00 to 2018-06-08 11:00:00
Freq: H
Empty DataFrame

In [754]:
full['Well_ft'] = data_well['Well_ft']
full['Tide_ft'] = data_tide['Tide_ft']
full['Rain_ft'] = data_rain['RAIN_FT']
full.index.name = 'DateHour'

In [755]:
full.Tide_ft.interpolate(inplace=True)
full.Tide_ft.fillna(method='ffill', inplace=True)
full.Tide_ft.fillna(method='bfill', inplace=True)


full.Rain_ft.interpolate(inplace=True)
full.Rain_ft.fillna(method='ffill', inplace=True)
full.Rain_ft.fillna(method='bfill', inplace=True)

full.Well_ft.interpolate(inplace=True)
full.Well_ft.fillna(method='ffill', inplace=True)
full.Well_ft.fillna(method='bfill', inplace=True)


In [757]:
assert not sum(full.Tide_ft.isna())
assert not sum(full.Rain_ft.isna())
assert not sum(full.Well_ft.isna())

In [758]:
new_file = "final_wells/" + file[0:len(file)-4].replace("-", "") + "_ALL.csv"

In [759]:
new_file

'final_wells/PB1680_ALL.csv'

In [760]:
full.to_csv(new_file, index=True)

# Incorporate Predicted Well Water Elevation After SAS Prediction

In [894]:
file = result[0]
well = file[:len(file)-4].replace("_", "").replace("-", "")
well_name = "Well " + file[:len(file)-9]
well_name
well

'F179FINAL'

In [895]:
pred = pd.read_sas("C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/" + "f179" + "_all_pred.sas7bdat")
full = pd.read_csv("C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/" + "F179" + "_ALL.csv", index_col='DateHour')
pred.set_index(full.index, inplace=True)

In [896]:
full[well_name + " Well Level"] = full.Well_ft
full[well_name + " Tide Level"] = full.Tide_ft
full[well_name + " Rain Level"] = full.Rain_ft
full[well_name + " Predicted Well Level"] = pred['FORECAST']
full[well_name + " Lower 95% Confidence Interval"] = pred.L95
full[well_name + " Upper 95% Confidence Interval"] = pred.U95

In [897]:
full.head()

,Well_ft,Tide_ft,Rain_ft,Well F179_ Well Level,Well F179_ Tide Level,Well F179_ Rain Level,Well F179_ Predicted Well Level,Well F179_ Lower 95% Confidence Interval,Well F179_ Upper 95% Confidence Interval
DateHour,,,,,,,,,
2007-10-01 01:00:00,3.1775,0.965,0.001771,3.1775,0.965,0.001771,NaN,NaN,NaN
2007-10-01 02:00:00,3.1900,0.358,0.002146,3.1900,0.358,0.002146,NaN,NaN,NaN
2007-10-01 03:00:00,3.2075,-0.144,0.005104,3.2075,-0.144,0.005104,NaN,NaN,NaN
2007-10-01 04:00:00,3.2275,-0.623,0.004792,3.2275,-0.623,0.004792,NaN,NaN,NaN
2007-10-01 05:00:00,3.2550,-0.994,0.002188,3.2550,-0.994,0.002188,3.288089,3.277932,3.298246


In [898]:
#full = full.iloc[:, 3:]
full.to_csv("C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/" + "F179" + "_FINAL.csv", index=True)

# Aggregate all data

In [807]:
import os
import glob
import pandas

path = 'C:/Users/Christopher/Documents/GitHub/Fall2_Homework/data/Well Data_Split/final_wells/'
extension = 'csv'
os.chdir(path)
result = [i for i in glob.glob('*.{}'.format(extension))]

term = result[1]
print(term)
print(term[(len(term)-9):len(term)])
result = [i for i in result if i[(len(i)-9):len(i)] == "FINAL.csv"]
print(result)

F179_FINAL.csv
FINAL.csv
['F179_FINAL.csv', 'F319_FINAL.csv', 'F45_FINAL.csv', 'G1220_FINAL.csv', 'G1260_FINAL.csv', 'G2147_FINAL.csv', 'G2866_FINAL.csv', 'G3549_FINAL.csv', 'G561_FINAL.csv', 'G580A_FINAL.csv', 'G860_FINAL.csv', 'PB1680_FINAL.csv']


In [851]:
df = pd.DataFrame(index=time_range)

In [869]:
well = pd.read_csv(path + result[1])

""
2007-10-01 01:00:00
2007-10-01 02:00:00
2007-10-01 03:00:00
2007-10-01 04:00:00
2007-10-01 05:00:00


In [870]:
well.head()

,DateHour,Well F-319 Well Level,Well F-319 Tide Level,Well F-319 Rain Level,Well F-319 Predicted Well Level,Well F-319 Lower 95% Confidence Interval,Well F-319 Upper 95% Confidence Interval
0,2007-10-01 01:00:00,3.05,0.965,0.005229,NaN,NaN,NaN
1,2007-10-01 02:00:00,3.05,0.358,0.004708,NaN,NaN,NaN
2,2007-10-01 03:00:00,3.06,-0.144,0.005083,NaN,NaN,NaN
3,2007-10-01 04:00:00,3.06,-0.623,0.005771,NaN,NaN,NaN
4,2007-10-01 05:00:00,3.09,-0.994,0.001708,3.131598,3.095427,3.167769


In [855]:
df['Well Level'] = well.iloc[:, 0]
df['Tide Level'] = well.iloc[:, 1]
df['Rain Level'] = well.iloc[:, 2]
df['Predicted Well Level'] = well.iloc[:,3]
df['Well Name'] = result[1]

In [873]:
df = pd.DataFrame(new_df)

In [874]:
df.head()

,DateHour,Predicted Well Level,Rain Level,Tide Level,Well Level,Well Name
0,2007-10-01 01:00:00,NaN,0.005229,0.965,3.05,F319
1,2007-10-01 02:00:00,NaN,0.004708,0.358,3.05,F319
2,2007-10-01 03:00:00,NaN,0.005083,-0.144,3.06,F319
3,2007-10-01 04:00:00,NaN,0.005771,-0.623,3.06,F319
4,2007-10-01 05:00:00,3.131598,0.001708,-0.994,3.09,F319


In [901]:
i=0
well = pd.read_csv(path + result[i])

new_df = {
    'DateHour': well.iloc[:, 0],
    'Well Level': well.iloc[:, 1],
    'Tide Level': well.iloc[:, 2],
    'Rain Level': well.iloc[:, 3],
    'Predicted Well Level': well.iloc[:,4],
    'Well Name': pd.Series([result[i][:len(result[i])-10] for name in range(len(well))])
}

df = df.append(pd.DataFrame(new_df))

In [904]:
df['Well Name'].unique()

array(['F319', 'F45', 'G1220', 'G1260', 'G2147', 'G2866', 'G3549', 'G561',
       'G580A', 'G860', 'PB1680', 'F179'], dtype=object)

In [905]:
df.to_csv(path + "Combined.csv", index=False)